In [1]:
import os
import shutil
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor

In [2]:
import sys
sys.path.append('../src')
from utils import run_cmd

In [3]:
bin_path = '/home/chen1i6c04/Tools/homopolish/homopolish.py'
mash_sketch = '/home/chen1i6c04/Tools/homopolish/bacteria.msh'
model = '/home/chen1i6c04/Tools/homopolish/R9.4.pkl'

In [7]:
def fn(infile, outdir, threads=4):
    os.makedirs(outdir, exist_ok=True)
    asm = os.path.join(outdir, 'contigs.fa')
    os.symlink(infile, asm)
    run_cmd(f"conda run -n homopolish python {bin_path} polish -a {asm} -s {mash_sketch} -m {model} -o {outdir} -t {threads}")

In [8]:
fn('/media/NAS/Central_Lab_Storage/MinION/mNGS/20210826/time_filt/denovo/barcode10_02h/contigs.fa',
   '/media/NAS/Central_Lab_Storage/MinION/mNGS/20210826/time_filt/denovo/barcode10_02h/homopolish')

In [5]:
dirpath = Path('/media/NAS/Central_Lab_Storage/MinION/mNGS/20210826/time_filt/denovo')

In [ ]:
with ProcessPoolExecutor(16) as executor:
    for i in dirpath.iterdir():
        infile = i/'contigs.fa'
        outdir = i/'homopolish'
        if infile.exists():
            executor.submit(fn, infile, outdir, 4)

In [6]:
for i in dirpath.iterdir():
    src = i/'homopolish'/'contigs_homopolished.fasta'
    dst = i/'contigs.fasta'
    if src.exists():
        shutil.copyfile(src, dst)
        shutil.rmtree(i/'homopolish')
    else:
        print(i)

/media/NAS/Central_Lab_Storage/MinION/mNGS/20210826/time_filt/denovo/barcode10_02h
